<a href="https://colab.research.google.com/github/akshaysaini1997/Deep-Learning/blob/master/NLP_project_(PS4_ecommerce_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [4]:
from google.colab import files
uploaded = files.upload()

Saving kaggle_ecommerce_complete_rank_crowd_agg_descriptions_all_rows.csv to kaggle_ecommerce_complete_rank_crowd_agg_descriptions_all_rows.csv


In [5]:
filename = 'kaggle_ecommerce_complete_rank_crowd_agg_descriptions_all_rows.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(filename)

# Display the first few rows of the DataFrame
df.head()

,unit_id,relevance,relevance_variance,product_image,product_link,product_price,product_title,query,rank,source,url,product_description
0,711158459,3.67,0.471,http://thumbs2.ebaystatic.com/d/l225/m/mzvzEUI...,http://www.ebay.com/itm/Sony-PlayStation-4-PS4...,$329.98,Sony PlayStation 4 (PS4) (Latest Model)- 500 G...,playstation 4,1,eBay,http://www.ebay.com/sch/i.html?_from=R40&_trks...,The PlayStation 4 system opens the door to an ...
1,711158460,4.00,0.000,http://thumbs3.ebaystatic.com/d/l225/m/mJNDmSy...,http://www.ebay.com/itm/Sony-PlayStation-4-Lat...,$324.84,Sony PlayStation 4 (Latest Model)- 500 GB Jet ...,playstation 4,2,eBay,http://www.ebay.com/sch/i.html?_from=R40&_trks...,The PlayStation 4 system opens the door to an ...
2,711158461,4.00,0.000,http://thumbs4.ebaystatic.com/d/l225/m/m10NZXA...,http://www.ebay.com/itm/Sony-PlayStation-4-PS4...,$324.83,Sony PlayStation 4 PS4 500 GB Jet Black Console,playstation 4,3,eBay,http://www.ebay.com/sch/i.html?_from=R40&_trks...,The PlayStation 4 system opens the door to an ...
3,711158462,3.67,0.471,http://thumbs2.ebaystatic.com/d/l225/m/mZZXTmA...,http://www.ebay.com/itm/Sony-PlayStation-4-500...,$350.00,Sony - PlayStation 4 500GB The Last of Us Rema...,playstation 4,4,eBay,http://www.ebay.com/sch/i.html?_from=R40&_trks...,NaN
4,711158463,3.33,0.471,http://thumbs3.ebaystatic.com/d/l225/m/mzvzEUI...,http://www.ebay.com/itm/Sony-PlayStation-4-PS4...,$308.00\nTrending at\n$319.99,Sony PlayStation 4 (PS4) (Latest Model)- 500 G...,playstation 4,5,eBay,http://www.ebay.com/sch/i.html?_from=R40&_trks...,The PlayStation 4 system opens the door to an ...


In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32671 entries, 0 to 32670
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   unit_id              32671 non-null  int64  
 1   relevance            20571 non-null  float64
 2   relevance_variance   20571 non-null  float64
 3   product_image        32671 non-null  object 
 4   product_link         32668 non-null  object 
 5   product_price        32671 non-null  object 
 6   product_title        32671 non-null  object 
 7   query                32671 non-null  object 
 8   rank                 32671 non-null  int64  
 9   source               32671 non-null  object 
 10  url                  32671 non-null  object 
 11  product_description  24800 non-null  object 
dtypes: float64(2), int64(2), object(8)
memory usage: 3.0+ MB


In [7]:
df.describe()

,unit_id,relevance,relevance_variance,rank
count,3.267100e+04,20571.000000,20571.000000,32671.000000
mean,7.119176e+08,3.255909,0.376791,22.133972
std,9.763701e+05,0.864320,0.391468,15.794443
min,7.111585e+08,1.000000,0.000000,1.000000
25%,7.111666e+08,2.670000,0.000000,9.000000
50%,7.111748e+08,3.670000,0.471000,18.000000
75%,7.131885e+08,4.000000,0.471000,34.000000
max,7.131966e+08,4.000000,1.470000,68.000000


In [8]:
df.isnull().sum()

unit_id                    0
relevance              12100
relevance_variance     12100
product_image              0
product_link               3
product_price              0
product_title              0
query                      0
rank                       0
source                     0
url                        0
product_description     7871
dtype: int64

In [9]:
df.fillna('', inplace=True)

In [10]:
df.isnull().sum()

unit_id                0
relevance              0
relevance_variance     0
product_image          0
product_link           0
product_price          0
product_title          0
query                  0
rank                   0
source                 0
url                    0
product_description    0
dtype: int64

In [11]:
def clean_text(text):
    if isinstance(text, str):  # check if text is a string
        text = text.lower()  # convert to lowercase
        text = re.sub(r'[^\w\s]', '', text)  # remove punctuation and special characters
        text = re.sub(r'\d+', '', text)  # remove numbers
    else:
        text = ''  # or some other default value you want to use for missing values
    return text

In [12]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

def tokenize_text(text):
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return ' '.join(tokens)

df['product_title'] = df['product_title'].apply(tokenize_text)
df['product_description'] = df['product_description'].apply(tokenize_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['product_description'])

In [14]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
token_count_matrix = count_vectorizer.fit_transform(df['product_description'])

lda = LatentDirichletAllocation(n_components=5, max_iter=5)
topics = lda.fit_transform(token_count_matrix)

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

sentiments = []
for text in df['product_description']:
    # Simple sentiment analysis using logistic regression
    sentiment = 1 if text.count('good') > text.count('bad') else 0
    sentiments.append(sentiment)

df['sentiment'] = sentiments

X_sentiment = vectorizer.transform(df['product_description'])
y_sentiment = df['sentiment']

log_reg = LogisticRegression()
log_reg.fit(X_sentiment, y_sentiment)

y_pred = log_reg.predict(X_sentiment)
print("Sentiment Analysis Accuracy:", accuracy_score(y_sentiment, y_pred))

Sentiment Analysis Accuracy: 0.9881546325487436


In [16]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier


In [17]:
# Sentiment Analysis
sentiments = []
for text in df['product_description']:
    # Simple sentiment analysis using logistic regression
    sentiment = 1 if text.count('good') > text.count('bad') else 0
    sentiments.append(sentiment)

df['sentiment'] = sentiments

X_sentiment = vectorizer.transform(df['product_description'])
y_sentiment = df['sentiment']

In [18]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_sentiment, y_sentiment, test_size=0.2, random_state=42)

In [19]:
# Define machine learning models
models = [
    LogisticRegression(),
    SVC(kernel='linear', C=1),
    RandomForestClassifier(n_estimators=100),
    MultinomialNB(),
    DecisionTreeClassifier(),
    KNeighborsClassifier(n_neighbors=5)
]

In [20]:
# Train and evaluate each model
for model in models:
    model_name = model.__class__.__name__
    print(f"Training {model_name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{model_name} Accuracy: {accuracy_score(y_test, y_pred):.3f}")
    print(f"{model_name} Classification Report:\n{classification_report(y_test, y_pred)}")
    print(f"{model_name} Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}\n")

Training LogisticRegression...
LogisticRegression Accuracy: 0.985
LogisticRegression Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6404
           1       0.94      0.24      0.39       131

    accuracy                           0.98      6535
   macro avg       0.96      0.62      0.69      6535
weighted avg       0.98      0.98      0.98      6535

LogisticRegression Confusion Matrix:
[[6402    2]
 [  99   32]]

Training SVC...
SVC Accuracy: 0.993
SVC Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6404
           1       0.94      0.71      0.81       131

    accuracy                           0.99      6535
   macro avg       0.97      0.85      0.90      6535
weighted avg       0.99      0.99      0.99      6535

SVC Confusion Matrix:
[[6398    6]
 [  38   93]]

Training RandomForestClassifier...
RandomForestClassifier Ac